In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda, Flatten
from keras.layers import Convolution1D, Dense, Dropout, MaxPooling1D, LSTM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from keras import callbacks
from keras.callbacks import CSVLogger
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn import metrics
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas  as pd
import numpy as np
import os
import time
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
        f1_score, classification_report,
        confusion_matrix, roc_curve,
        roc_auc_score, accuracy_score,
        log_loss)
from sklearn import __version__ as sklearn_version
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import NearMiss
from datetime import datetime
np.random.seed(0)

In [3]:
dir2='/content/drive/MyDrive/8thSemProject/botnet_dataset/device2/'
device2_df=pd.DataFrame()
i=0
for file in os.listdir(dir2+'gafgyt/'):
  df_file = pd.read_csv(os.path.join(dir2+'gafgyt/', file), encoding = 'ISO-8859-1',low_memory=False)
  df_file["label"]=i
  i=i+1
  device2_df=pd.concat([device2_df, df_file])
for file in os.listdir(dir2+'mirai/'):
  df_file = pd.read_csv(os.path.join(dir2+'mirai/', file), encoding = 'ISO-8859-1',low_memory=False)
  df_file["label"]=i
  i=i+1
  device2_df=pd.concat([device2_df, df_file])
df_file=pd.read_csv('/content/drive/MyDrive/8thSemProject/botnet_dataset/device2/benign.csv')
df_file['label']=i
device2_df=pd.concat([device2_df, df_file])

In [4]:
x=device2_df.drop(['label'],axis=1)
y=device2_df['label']
normalized_df=(x-x.mean())/x.std()
y.head()

0    0
1    0
2    0
3    0
4    0
Name: label, dtype: int64

In [5]:
X_train, X_test, y_train, y_test = train_test_split(normalized_df, y, test_size=0.2, random_state=0)
y_train = np.array(y_train)

y_train = np.array(y_train)
y_test = np.array(y_test)
X_train = np.array(X_train)
X_test = np.array(X_test)
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1]))
print('xtrain={}, ytrain={}, xtest={}, ytest={}'.format(X_train.shape,y_train.shape,X_test.shape,y_test.shape))

xtrain=(668700, 115), ytrain=(668700,), xtest=(167176, 115), ytest=(167176,)


In [6]:
!pip install xgboost

In [7]:
from xgboost import XGBClassifier
model = XGBClassifier(learning_rate=0.02,
    n_estimators=10,
    nthread=3,
    tree_method="gpu_hist")
st=time.time()
model.fit(X_train, y_train)
en=time.time()
print(en-st)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [04:31:44] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


13.879132986068726


In [8]:
y_pred=model.predict(X_test)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [04:31:47] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [04:31:47] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [9]:
y_train.shape

(668700,)

In [10]:
from sklearn import metrics
from sklearn.metrics import f1_score
print('XGBoost Classifier')

print('Accuracy = ', metrics.accuracy_score(y_test, y_pred)*100)
print("Confusion Matrix =\n", metrics.confusion_matrix(y_test, y_pred, labels=None,
                                              sample_weight=None))
print("Recall =", metrics.recall_score(y_test, y_pred, labels=None,
                                             pos_label=1, average='weighted',
                                             sample_weight=None))
print("Classification Report =\n", metrics.classification_report(y_test, y_pred,
                                                                 labels=None,
                                                                 target_names=None,
                                                                 sample_weight=None,
                                                                 digits=2,
                                                                 output_dict=False))

print("F1 Score = ",f1_score(y_test, y_pred, average='macro'))

XGBoost Classifier
Accuracy =  99.94377183327751
Confusion Matrix =
 [[10735     6     1     1     0     0     0     0     0     0     0]
 [   42  5891     2     2     0     0     0     0     0     0     0]
 [    0     1  5541     0     1     0     0     0     0     0     0]
 [    1     2     0 18896     0     0     0     0     0     0     0]
 [    0     0     0     0 20951     0     0     0     0     0     0]
 [    0     0     0     0     0 22724     0     0     1     0     0]
 [    0     0     0     0     0     0  8620     0     0     0     0]
 [    0     0     0     0     0     0     0 23492     0     0     0]
 [    0     0     0     2     0     0     0     0 30258    10     0]
 [    0     0     0     0     0     0     0     0     1 17355     0]
 [    0    12     0     0     6     0     0     1     0     2  2619]]
Recall = 0.9994377183327751
Classification Report =
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     10743
         

In [11]:
def rf_classifier(data, device_name, scaling=False,):
    X = data.drop(['label'], axis=1)
    y = data['label']
    print(f'Original Shape:{X.shape}, {y.shape}')
    if scaling == False:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=47)
        model_name = f'{device_name}_without_scaling_unbalanced_model.pkl'
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=47)
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        model_name = f'{device_name}_with_scaling_unbalanced_model.pkl'

    clf = RandomForestClassifier()
    start=time.time()
    model_res = clf.fit(X_train, y_train)
    end=time.time()
    tt=end-start
    y_pred = model_res.predict(X_test)
    y_pred_prob = model_res.predict_proba(X_test)
    lr_probs = y_pred_prob[:,1]
    ac = accuracy_score(y_test, y_pred)
    importances = pd.DataFrame({'feature':X.columns,'importance':np.round(clf.feature_importances_,3)})
    importances = importances.sort_values('importance',ascending=False).set_index('feature')

    f1 = f1_score(y_test, y_pred, average='weighted')
    cm = confusion_matrix(y_test, y_pred)
    print("Random Forest Classifier")
    print(f"Training time: {tt}s")
    print('Accuracy=%.3f' % (ac))
    print('f1-score=%.3f' % (f1))
    print('Confusion Matrix = \n', cm)
    print('Classification Report = \n',classification_report(y_test, y_pred))

In [12]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier(device2_df,'device1',True)

Original Shape:(835876, 115), (835876,)
Random Forest Classifier
Training time: 208.92313742637634s
Accuracy=0.887
f1-score=0.848
Confusion Matrix = 
 [[15995     3     0     1     0     0     0     0     0     0     0]
 [    0  9071     0     1     0     0     0     0     0     0     0]
 [    0     2  8271     1     0     0     0     0     0     0     0]
 [    0     1     0    23 28324     0     0     0     0     0     0]
 [    0     1     0     2 31129     0     0     0     0     0     0]
 [    0     0     0     0     0 34101     0     0     0     0     0]
 [    0     0     0     0     0     0 13081     0     1     0     0]
 [    0     0     0     0     0     0     0 35079     0     0     0]
 [    0     0     0     0     0     0     0     0 45521     0     0]
 [    0     0     0     0     0     0     0     0     0 26213     0]
 [    0     0     0     0     0     0     0     0     0     0  3942]]
Classification Report = 
               precision    recall  f1-score   support

        

In [13]:
from sklearn import metrics
from sklearn.metrics import f1_score
def knn_classifier_unbalanced(data, scaling=False):
    X = data.drop(['label'], axis=1)
    y = data['label']
    print(f'Original Shape:{X.shape}, {y.shape}')
    test_scores = []
    train_scores = []
    error = []
    if scaling == False:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=47)
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=47)
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

    knn = KNeighborsClassifier(3)
    start=time.time()
    knn.fit(X_train,y_train)
    end=time.time()
    tt=end-start
    y_pred = knn.predict(X_test)
    error.append(np.mean(y_pred != y_test))
    train_scores.append(knn.score(X_train,y_train))
    test_scores.append(knn.score(X_test,y_test))
    print("KNN")
    print(f"Training time: {tt}s")
    print("Recall =", metrics.recall_score(y_test, y_pred, labels=None,
                                             pos_label=1, average='weighted',
                                             sample_weight=None))
    print('Classification Report = \n',classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print(f'Accuracy on Training set:{train_scores}')
    print(f'Accuracy on Test set:{test_scores}')

In [ ]:
knn_classifier_unbalanced(device2_df,True)

Original Shape:(835876, 115), (835876,)


In [ ]:
def dt_classifier(data, device_name, scaling=False,):
    X = data.drop(['label'], axis=1)
    y = data['label']
    print(f'Original Shape:{X.shape}, {y.shape}')
    if scaling == False:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=47)
        model_name = f'{device_name}_without_scaling_unbalanced_model_decision_tree_gini.pkl'
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=47)
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        model_name = f'{device_name}_with_scaling_unbalanced_model_decision_tree_gini.pkl'

    clf = DecisionTreeClassifier(criterion='gini', random_state= 1234, max_depth  = 5 )
    start=time.time()
    model_res = clf.fit(X_train, y_train)
    end=time.time()
    tt=end-start
    y_pred = model_res.predict(X_test)
    y_pred_prob = model_res.predict_proba(X_test)
    lr_probs = y_pred_prob[:,1]
    ac = accuracy_score(y_test, y_pred)
    importances = pd.DataFrame({'feature':X.columns,'importance':np.round(clf.feature_importances_,3)})
    importances = importances.sort_values('importance',ascending=False).set_index('feature')
    f1 = f1_score(y_test, y_pred, average='weighted')
    cm = confusion_matrix(y_test, y_pred)
    print('Decision tree')
    print("Recall =", metrics.recall_score(y_test, y_pred, labels=None,
                                             pos_label=1, average='weighted',
                                             sample_weight=None))
    print(f"Training time: {tt}s")
    print('Accuracy=%.3f' % (ac))
    print('f1-score=%.3f' % (f1))
    print('Confusion Matrix = \n', cm)
    print('Classification Report',classification_report(y_test, y_pred))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt_classifier(device2_df,'device2',True)